# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## GPU Check

First, you'll need to enable GPUs for the notebook:

- Navigate to `Edit`→`Notebook Settings`
- Select T4 GPU from the Hardware Accelerator section
- Click `Save` and accept.

Next, we'll confirm that we can connect to the GPU:

In [1]:
import torch

if not torch.cuda.is_available():
    raise RuntimeError("GPU not available")
else:
    print("GPU is available!")

GPU is available!


## Installing dependencies

In [ ]:
%pip install --quiet https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu122/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl

In [ ]:
%pip install --quiet ragatouille PyPDF2 structured-qa

In [ ]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/main/benchmark/structured_qa.csv

# Setup

In [6]:
import os

os.environ["LOGURU_LEVEL"] = "INFO"

In [7]:
from loguru import logger

In [8]:
import PyPDF2


def load_pdf(pdf_file: str) -> str | None:
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return "\n".join(page.extract_text() for page in pdf_reader.pages)
    except Exception as e:
        logger.exception(e)
        return None

## Function to Process all questions for a single Document

In [9]:
from ragatouille import RAGPretrainedModel
from ragatouille.data import CorpusProcessor


ANSWER_WITH_TYPE_PROMPT = """
You are a rigorous assistant answering questions.
You must only answer based on the current information available which is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return "I need more info" srting and nothing else:

If the current information is enough to answer, you must return one of the following formats:
- YES/NO (for boolean questions)
- Number (for numeric questions)
- Single letter (for multiple-choice questions)
"""


def process_document(
    document_file,
    document_data,
    model,
):
    logger.info("Setting up RAG")
    RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
    corpus_processor = CorpusProcessor()
    documents = corpus_processor.process_corpus([load_pdf(document_file)])
    RAG.encode([x["content"] for x in documents])

    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        question = row["question"]
        question_part, *options = question.split("?")

        logger.info(f"Question: {question}")
        results = RAG.search_encoded_docs(query=question_part, k=3)
        current_info = "\n".join(result["content"] for result in results)
        logger.info(current_info[:100])

        messages = [
            {
                "role": "system",
                "content": ANSWER_WITH_TYPE_PROMPT.format(CURRENT_INFO=current_info),
            },
            {"role": "user", "content": question},
        ]
        answer = model.get_response(messages)
        logger.info(answer)
        answers[index] = answer
        sections[index] = None

    return answers, sections

## Load Model

In [10]:
from structured_qa.model_loaders import load_llama_cpp_model

In [ ]:
model = load_llama_cpp_model(
    "bartowski/Qwen2.5-7B-Instruct-GGUF/Qwen2.5-7B-Instruct-Q8_0.gguf"
)

# Run Benchmark

In [ ]:
from pathlib import Path
from urllib.request import urlretrieve

import pandas as pd

logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)
for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    if not Path(downloaded_document).exists():
        urlretrieve(document_link, downloaded_document)
        logger.info(f"Downloaded {document_link} to {downloaded_document}")
    else:
        logger.info(f"File {downloaded_document} already exists")

    answers, sections = process_document(downloaded_document, document_data, model)

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

In [13]:
results = pd.read_csv("results.csv")
for index, result in results.iterrows():
    if result["pred_answer"].startswith(
        (f"-{result['answer']}", f"{result['answer']}")
    ):
        results.loc[index, "pred_answer"] = result["answer"]
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,type,section,question,answer,pred_answer,pred_section
2,2,https://arxiv.org/pdf/1706.03762,Scientific Paper,3.1 Encoder and Decoder Stacks,How many layers compose the decoder?,6,N=6,NaN
10,10,https://arxiv.org/pdf/1706.03762,Scientific Paper,5.4 Regularization,What was the dropout rate used for the base mo...,0.1,PDROP= 0.1,NaN
28,28,https://arxiv.org/pdf/2201.11903,Scientific Report,3.1 Experimental Setup,How many large language models were evaluated?,5,FIVE,NaN
32,32,https://arxiv.org/pdf/2201.11903,Scientific Report,5 Symbolic Reasoning,Which symbolic reasoning task is used as an ou...,A,I NEED MORE INFO,NaN
34,34,https://arxiv.org/pdf/2201.11903,Scientific Report,3.2 Results,How many random samples were examined to under...,100,50,NaN
37,37,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE EFFECTS,How many different races are there?,6,I NEED MORE INFO,NaN
41,41,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CHAPTER OVERVIEW,"After taking a landmark tile, do you reveal a ...",NO,YES,NaN
42,42,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE COSTS,Can a player pay coins to compensate for missi...,YES,NO,NaN
45,45,https://github.com/mozilla-ai/structured-qa/re...,Board Game,CARD AND TILE EFFECTS,Which type of cards provide coins? -A: Gray -B...,B,I NEED MORE INFO,NaN
55,55,https://github.com/mozilla-ai/structured-qa/re...,Board Game,EXPEDITION PHASE,Do you need a fish to conquer a distant island?,YES,NO,NaN


In [14]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.8349514563106796